### Can we classify each phase as relatively low or high anxiety for each subject? ###
#### APD, WESAD ####

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    # train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

model_phases_wesad = dr_w.Phases.PHASE_ORDER

# anxiety_label_type = "Anxiety"
anxiety_label_type = None
wesad_label_type = "stai"

models = {
    # "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "RF": RandomForestClassifier(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

threshold = "fixed"
test_size = 1.0

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [33]:
# TRAIN ON APD AND TEST ON WESAD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
# drop subjects with noisy data
x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"] + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.0:
        print(f"Model evaluation metrics for {model_name}:")
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
print("\n")

Ratio of positive to negative labels (0.2857142857142857) is under 0.333, oversampling positive class.
Ratio of positive to negative labels (0.2727272727272727) is under 0.333, oversampling positive class.
y_train:
0    315
1    104
Name: label, dtype: int64
y_test:
0    55
1    18
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [45 28]
Model RF, Predictions: [0 1], [15 58]
Model XGB, Predictions: [0 1], [12 61]

[('lf_hf_ratio', 1.5303892390759992), ('lf_rr', 1.4425205632727884), ('mean_SCL', 0.9828521184983557), ('hf_rr', 0.89251784042838), ('sdnn', 0.19839299462867824), ('bpm', 0.0633231251665666), ('rmssd', -0.1393740498186946), ('SCR_rate', -0.45640706993009156)]


[('mean_SCL', 0.1445832262284716), ('bpm', 0.13699012335797692), ('lf_rr', 0.13381469081260253), ('hf_rr', 0.1268899505142711), ('SCR_rate', 0.12669011646776782), ('lf_hf_ratio', 0.1209120788504522), ('sdnn', 0.10711759242217438), ('rmssd', 0.1030022213462834)]


[('bpm', 0.15064101), ('mean_SCL', 0.14778303

In [34]:
# TRAIN ON APD AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for j, phases_wesad in enumerate(model_phases_wesad):
    phases_wesad = [phases_wesad]
    print(f"WESAD PHASES {phases_wesad} " + "-"*50)
    x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
    x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
    # drop subjects with noisy data
    x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

    if anxiety_label_type is not None:
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    acc_results = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    reports = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    num_iters = 1
    get_importance = True
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])
            if get_importance:
                try:
                    print("")
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
                except Exception as e:
                    print(e)
                    # print(out[model_name][0][2])
                print("")

    for model_name in acc_results.keys():
        acc = np.mean(acc_results[model_name])
        print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
        if acc > 0.0:
            print(f"Model evaluation metrics for {model_name}:")
            p = np.mean([report["precision"] for report in reports[model_name]])
            r = np.mean([report["recall"] for report in reports[model_name]])
            f1 = np.mean([report["f1"] for report in reports[model_name]])
            auc = np.mean([report["auc"] for report in reports[model_name]])
            report = reports[model_name]
            print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    print("\n")

WESAD PHASES ['Base'] --------------------------------------------------
Ratio of positive to negative labels (0.2857142857142857) is under 0.333, oversampling positive class.
y_train:
0    315
1    104
Name: label, dtype: int64
y_test:
0    14
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [ 2 12]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Model RF, Predictions: [0 1], [7 7]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Model XGB, Predictions: [0 1], [ 2 12]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.

[('lf_hf_ratio', 1.2867678430166132), ('lf_rr', 1.235952373024034), ('mean_SCL', 1.0093544849716654), ('hf_rr', 0.8329653873283106), ('sdnn', 0.2197112292886464), ('rmssd', 0.011762822945096535), ('bpm', -0.053931275713352426), ('SCR_rate', -0.24849161852344506)]


[('mean_SCL', 0.14757216597124348), ('b

In [40]:
# TRAIN ON WESAD AND TEST ON APD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
# x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
if anxiety_label_type is not None:
    x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, is_resample=True, test_size=test_size, save_metrics=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.0:
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Report:\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}")
print("\n")

Ratio of positive to negative labels (0.2727272727272727) is under 0.333, oversampling positive class.
Ratio of positive to negative labels (0.2857142857142857) is under 0.333, oversampling positive class.
y_train:
0    55
1    18
Name: label, dtype: int64
y_test:
0    315
1    104
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [408  11]
0.6363636363636364
0.0673076923076923
0.12173913043478261
Model RF, Predictions: [0 1], [313 106]
0.2358490566037736
0.2403846153846154
0.2380952380952381
Model XGB, Predictions: [0 1], [321  98]
0.17346938775510204
0.16346153846153846
0.1683168316831683

[('bpm', 2.226634560138877), ('mean_SCL', 1.4167519240319062), ('SCR_rate', 0.9402132778054553), ('lf_rr', 0.7477350081305932), ('rmssd', 0.28534748709682756), ('hf_rr', 0.09245063929945856), ('sdnn', -0.3665490309906174), ('lf_hf_ratio', -0.6929390214573399)]


[('bpm', 0.2054819872820546), ('mean_SCL', 0.18623954929849196), ('hf_rr', 0.18585391989543326), ('sdnn', 0.10987040703146236), 

In [37]:
# TRAIN ON WESAD AND TEST ON APD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for j, phases_apd in enumerate(model_phases_apd):
    print(f"APD PHASES {phases_apd} " + "-"*50)
    phases_apd = [phases_apd]
    x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
    x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
    # drop subjects with noisy data
    x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    if anxiety_label_type is not None:
        x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    acc_results = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    reports = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    num_iters = 1
    get_importance = True
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, is_resample=True, test_size=test_size, save_metrics=True, get_importance=get_importance, drop_subject=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])
            if get_importance:
                try:
                    print("")
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
                except Exception as e:
                    print(e)
                    # print(out[model_name][0][2])
                print("")

    for model_name in acc_results.keys():
        acc = np.mean(acc_results[model_name])
        print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
        if acc > 0.0:
            p = np.mean([report["precision"] for report in reports[model_name]])
            r = np.mean([report["recall"] for report in reports[model_name]])
            f1 = np.mean([report["f1"] for report in reports[model_name]])
            auc = np.mean([report["auc"] for report in reports[model_name]])
            report = reports[model_name]
            print(f"Report:\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}")
    print("\n")

APD PHASES Baseline_Rest --------------------------------------------------
Ratio of positive to negative labels (0.2727272727272727) is under 0.333, oversampling positive class.
Ratio of positive to negative labels (0.125) is under 0.333, oversampling positive class.
y_train:
0    55
1    18
Name: label, dtype: int64
y_test:
0    40
1    13
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [49  4]
Model RF, Predictions: [0 1], [39 14]
Model XGB, Predictions: [0 1], [35 18]

[('bpm', 2.4890214635260905), ('mean_SCL', 1.102627180166507), ('SCR_rate', 0.8078992189750194), ('lf_rr', 0.6742538289830456), ('rmssd', 0.5276468053227578), ('hf_rr', 0.08900204680800797), ('sdnn', -0.30009475200652397), ('lf_hf_ratio', -0.7170581552624832)]


[('bpm', 0.28410011768139454), ('hf_rr', 0.1613034106595782), ('mean_SCL', 0.15180835691260097), ('rmssd', 0.09282600146315305), ('SCR_rate', 0.08943303665840778), ('lf_rr', 0.07712379676257473), ('sdnn', 0.07678012125601244), ('lf_hf_ratio', 0.06